# Anonymeter example notebook

This example notebook demonstrates the usage of `Anonymeter`, a software to derive GDPR-aligned measures of the privacy of synthetic datasets in an empirical, attack based fashion.

`Anonymeter` contains privacy evaluators which measures the risks of singling out, linkability, and inference which might incur to data donors following the release of synthetic dataset. These risk are the three key indicators of factual anonymization according to the European General Data Protection Regulation (GDPR). For more details, please refer to [M. Giomi et al. 2022](https://petsymposium.org/popets/2023/popets-2023-0055.php).

### Basic usage pattern

For each of these privacy risks anonymeter provide an `Evaluator` class. The high-level classes `SinglingOutEvaluator`, `LinkabilityEvaluator`, and `InferenceEvaluator` are the only thing that you need to import from `Anonymeter`.

Despite the different nature of the privacy risks they evaluate, these classes have the same interface and are used in the same way. To instantiate the evaluator you have to provide three dataframes: the original dataset `ori` which has been used to generate the synthetic data, the synthetic data `syn`, and a `control` dataset containing original records which have not been used to generate the synthetic data. 

Another parameter common to all evaluators is the number of target records to attack (`n_attacks`). A higher number will reduce the statistical uncertainties on the results, at the expense of a longer computation time.

```python
evaluator = *Evaluator(ori: pd.DataFrame, 
                       syn: pd.DataFrame, 
                       control: pd.DataFrame,
                       n_attacks: int)
```

Once instantiated the evaluation pipeline is executed when calling the `evaluate`, and the resulting estimate of the risk can be accessed using the `risk()` method.

```python
evaluator.evaluate()
risk = evaluator.risk()
```

### A peak under the hood

In `Anonymeter` each privacy risk is derived from a privacy attacker whose task is to use the synthetic dataset to come up with a set of *guesses* of the form:
- "there is only one person with attributes X, Y, and Z" (singling out)
- "records A and B belong to the same person" (linkability)
- "a person with attributes X and Y also have Z" (inference)

Each evaluation consists of running three different attacks:
- the "main" privacy attack, in which the attacker uses the synthetic data to guess information on records in the original data.
- the "control" privacy attack, in which the attacker uses the synthetic data to guess information on records in the control dataset. 
- the "baseline" attack, which models a naive attacker who ignores the synthetic data and guess randomly.

Checking how many of these guesses are correct, the success rates of the different attacks are measured and used to derive an estimate of the privacy risk. In particular, the "control attack" is used to separate what the attacker learns from the *utility* of the synthetic data, and what is instead indication of privacy leaks. The "baseline attack" instead functions as a sanity check. The "main attack" attack should outperform random guessing in order for the results to be trusted. 

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from anonymeter.evaluators import SinglingOutEvaluator
from anonymeter.evaluators import LinkabilityEvaluator
from anonymeter.evaluators import InferenceEvaluator

## Downloading the data

For this example, we will use the famous `Adults` (more details [here](https://archive.ics.uci.edu/ml/datasets/adult)) dataset. This dataset contains aggregated census data, where every row represent a population segment. For the purpose of demonstrating `Anonymeter`, we will use this data as if each row would in fact refer to a real individual. 

The synthetic version has been generated by `CTGAN` from [SDV](https://sdv.dev/SDV/user_guides/single_table/ctgan.html), as explained in the paper accompanying this code release. For details on the generation process, e.g. regarding hyperparameters, see Section 6.2.1 of [the accompanying paper](https://petsymposium.org/popets/2023/popets-2023-0055.php)).

We pull these datasets from the [Statice](https://www.statice.ai/) public GC bucket:

In [28]:
# bucket_url = "https://storage.googleapis.com/statice-public/anonymeter-datasets/"

# ori = pd.read_csv('~/downloads/small_data/real.csv')
# syn = pd.read_csv('~/downloads/small_data/dpdffution_tabsyn.csv')
# control = pd.read_csv('~/downloads/small_data/test.csv')
# dif_syn = pd.read_csv('~/downloads/small_data/diffusion_tabsyn.csv')
syn_dp = pd.read_csv(r"C:\Users\26069\Downloads\ctgan_combined-1.csv")
ori = pd.read_csv(r'C:\Users\26069\Downloads\train_data_after_feature_selection.csv')
control = pd.read_csv(r'C:\Users\26069\Downloads\test_data_after_feature_selection.csv')
#no dp
#syn = pd.read_csv('~/downloads/161/balanced_dataset_no_dp.csv')

In [29]:
ori = ori.sample(n=50000, random_state=0)
syn_dp = syn_dp.sample(n=50000, random_state=0)

In [30]:
syn_dp.shape

(50000, 24)

In [31]:
ori.shape

(50000, 23)

In [32]:
control.shape

(23784, 23)

In [33]:
ori.head()

,city,series_dev,emui_dev,device_name,device_size,net_type,creat_type_cd,slot_id,spread_app_id,app_second_class,...,task_id_count,adv_id_count,user_id_task_id_nunique,user_id_adv_prim_id_nunique,user_id_slot_id_nunique,user_id_spread_app_id_nunique,age_task_id_nunique,age_adv_id_nunique,gender_task_id_nunique,label
3519005,213,34,20,137,2032,7,8,38,162,23,...,412,412,81,27,13,16,7019,7834,10244,0
2291869,372,31,21,151,2117,7,8,59,162,14,...,1939,1939,92,37,12,17,7019,7834,6898,0
2675847,249,30,28,278,1555,7,8,50,174,18,...,942,942,100,37,14,15,7019,7834,10244,0
1149062,186,32,32,187,1186,7,8,67,336,17,...,8,3,70,22,14,11,8363,9358,10244,0
3010903,263,30,35,113,1983,6,8,17,181,20,...,53,53,45,18,9,8,7845,8730,10244,0


In [34]:
syn_dp = syn_dp.drop(columns=['Unnamed: 0'])
syn_dp.head()

,city,series_dev,emui_dev,device_name,device_size,net_type,creat_type_cd,slot_id,spread_app_id,app_second_class,...,task_id_count,adv_id_count,user_id_task_id_nunique,user_id_adv_prim_id_nunique,user_id_slot_id_nunique,user_id_spread_app_id_nunique,age_task_id_nunique,age_adv_id_nunique,gender_task_id_nunique,label
2558159,344,32,20,229,2032,7,8,65,199,17,...,701,701,78,18,15,9,4321,4669,10244,0
2568557,354,31,20,346,2103,6,7,30,213,23,...,8543,8543,33,17,11,10,8065,9014,10244,0
368826,283,34,21,244,2383,6,8,12,246,23,...,5527,5527,83,38,18,17,7845,8730,10244,0
442604,431,11,32,301,2401,7,7,25,213,23,...,2129,779,53,31,14,10,8065,9014,10244,0
2046037,319,27,11,229,2032,7,8,65,347,20,...,84,84,14,9,12,8,7637,8530,8093,0


In [40]:

from diffprivlib.models import GaussianNB

# Define the features and labels for the syn_dp dataset
X_syn_dp = syn_dp.drop(columns=['label'])
y_syn_dp = syn_dp['label']

# No scaling applied as per the previous approach
X_syn_dp_scaled = X_syn_dp

# Apply Differential Privacy to model training on syn_dp
clf_syn_dp = GaussianNB(epsilon=0.003)  # Adjust epsilon for desired privacy level
clf_syn_dp.fit(X_syn_dp_scaled, y_syn_dp)

# Evaluate the model on the syn_dp dataset
print("DP GaussianNB Test Accuracy on syn_dp:", clf_syn_dp.score(X_syn_dp_scaled, y_syn_dp))


DP GaussianNB Test Accuracy on syn_dp: 0.54328


C:\Users\26069\anaconda3\envs\myenv3.11\Lib\site-packages\diffprivlib\models\naive_bayes.py:107: PrivacyLeakWarning: Bounds have not been specified and will be calculated on the data provided. This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify bounds for each dimension.
  warnings.warn("Bounds have not been specified and will be calculated on the data provided. This will "


In [41]:
syn_dp_with_dp = X_syn_dp_scaled.copy()
syn_dp_with_dp['label'] = y_syn_dp
syn_dp = syn_dp_with_dp
print(syn_dp.head()) 

         city  series_dev  emui_dev  device_name  device_size  net_type  \
2558159   344          32        20          229         2032         7   
2568557   354          31        20          346         2103         6   
368826    283          34        21          244         2383         6   
442604    431          11        32          301         2401         7   
2046037   319          27        11          229         2032         7   

         creat_type_cd  slot_id  spread_app_id  app_second_class  ...  \
2558159              8       65            199                17  ...   
2568557              7       30            213                23  ...   
368826               8       12            246                23  ...   
442604               7       25            213                23  ...   
2046037              8       65            347                20  ...   

         task_id_count  adv_id_count  user_id_task_id_nunique  \
2558159            701           701         

As visible the dataset contains several demographic information, as well as information regarding the education, financial situation, and personal life of some tens of thousands of "individuals".

### Measuring the singling out risk

The `SinglingOutEvaluator` try to measure how much the synthetic data can help an attacker finding combination of attributes that single out records in the training data. 

With the following code we evaluate the robustness of the synthetic data to "univariate" singling out attacks, which try to find unique values of some attribute which single out an individual. 


##### NOTE:

The `SingingOutEvaluator` can sometimes raise a `RuntimeError`. This happens when not enough singling out queries are found. Increasing `n_attacks` will make this condition less frequent and the evaluation more robust, although much slower.


In [42]:
#with dp
evaluator2 = SinglingOutEvaluator(ori=ori, 
                                 syn=syn_dp, 
                                 control=control,
                                 n_attacks=500)

try:
    evaluator2.evaluate(mode='univariate')
    risk = evaluator2.risk()
    print(risk)

except RuntimeError as ex: 
    print(f"Singling out evaluation failed with {ex}. Please re-run this cell."
          "For more stable results increase `n_attacks`. Note that this will "
          "make the evaluation slower.")

PrivacyRisk(value=0.0019923464830288893, ci=(0.0, 0.00865926688774084))


In [43]:
res2 = evaluator2.results()

print("Successs rate of main attack:", res2.attack_rate)
print("Successs rate of baseline attack:", res2.baseline_rate)
print("Successs rate of control attack:", res2.control_rate)

Successs rate of main attack: SuccessRate(value=0.005796921549853016, error=0.005443785155293741)
Successs rate of baseline attack: SuccessRate(value=0.0038121702307761206, error=0.00381217023077612)
Successs rate of control attack: SuccessRate(value=0.0038121702307761206, error=0.00381217023077612)


The risk estimate is accompanied by a confidence interval (at 95% level by default) which accounts for the finite number of attacks performed, 500 in this case. 

Using the `queries()` method, we can see what kind of singling out queries (i.e. the *guesses*) the attacker has come up with:

As visible it was able to pick up the `fnlwgt` has many (~63%) unique integer values  and that it can provide a powerful handle for singling out. This should result in a singling out risk which is *compatible* within the confidence level with a few percentage points. The actual results can vary depending on notebook execution. 

### Checking singling out with multivariate predicates

The `SinglingOutEvaluator` can also attack the dataset using predicates which are combining different attributes. These are the so called `multivariate` predicates. 

To run the analysis using the `multivariate` singling out attack, the `mode` parameter of `evaluate` needs to be set correctly. The number of attributes used in the attacker queries via the `n_cols` parameter, set to 4 in this example. 

In [44]:
#with dp diffusion
evaluator3 = SinglingOutEvaluator(ori=ori, 
                                 syn=syn_dp, 
                                 control=control,
                                 n_attacks=100, # this attack takes longer
                                 n_cols=4)


try:
    evaluator3.evaluate(mode='multivariate')
    risk = evaluator3.risk()
    print(risk)

except RuntimeError as ex: 
    print(f"Singling out evaluation failed with {ex}. Please re-run this cell."
          "For more stable results increase `n_attacks`. Note that this will "
          "make the evaluation slower.")

PrivacyRisk(value=0.0, ci=(0.0, 0.031447047755298066))


In [45]:
res3 = evaluator3.results()

print("Successs rate of main attack:", res3.attack_rate)
print("Successs rate of baseline attack:", res3.baseline_rate)
print("Successs rate of control attack:", res3.control_rate)

Successs rate of main attack: SuccessRate(value=0.08590720422900384, error=0.05158794316173117)
Successs rate of baseline attack: SuccessRate(value=0.01849674910349284, error=0.01849674910349284)
Successs rate of control attack: SuccessRate(value=0.14743468639684537, error=0.06688312844351454)


In [46]:
evaluator3.queries()[:3]

['user_id_count>= 649 & city<= 135 & gender_task_id_nunique<= 6898 & slot_id>= 58',
 'device_size<= 1088 & user_id_adv_prim_id_nunique<= 7 & device_name>= 336 & age_adv_id_nunique<= 7834',
 'series_dev<= 11 & u_feedLifeCycle<= 11 & app_second_class<= 13 & net_type<= 6']

In [47]:
# # without dp
# evaluator4 = SinglingOutEvaluator(ori=ori, 
#                                  syn=syn, 
#                                  control=control,
#                                  n_attacks=100, # this attack takes longer
#                                  n_cols=4)


# try:
#     evaluator4.evaluate(mode='multivariate')
#     risk = evaluator4.risk()
#     print(risk)
#     #print(evaluator6.queries()[:3])

# except RuntimeError as ex: 
#     print(f"Singling out evaluation failed with {ex}. Please re-run this cell."
#           "For more stable results increase `n_attacks`. Note that this will "
#           "make the evaluation slower.")


In [48]:
# res4 = evaluator4.results()

# print("Successs rate of main attack:", res4.attack_rate)
# print("Successs rate of baseline attack:", res4.baseline_rate)
# print("Successs rate of control attack:", res4.control_rate)

In [49]:
# evaluator4.queries()[:3]

# Measuring the Linkability risk

The `LinkabilityEvaluator` allows one to know how much the synthetic data will help an adversary who tries to link two other datasets based on a subset of attributes. 

For example, suppose that the adversary finds dataset A containing, among other fields, information about the profession and education of people, and dataset B containing some demographic and health related information. Can the attacker use the synthetic dataset to link these two datasets?

To run the `LinkabilityEvaluator` one needs to specify which columns of auxiliary information are available to the attacker, and how they are distributed between the two datasets A and B. This is done using the `aux_cols` parameter.

In [50]:
#aux_cols = [
#    ['type_employer', 'education', 'hr_per_week', 'capital_loss', 'capital_gain'],
#    [ 'race', 'sex', 'fnlwgt', 'age', 'country']
#]

#with dp
aux_cols = ["city", "spread_app_id", "u_refreshTimes", "user_id_count"]

evaluator5 = LinkabilityEvaluator(ori=ori, 
                                 syn=syn_dp, 
                                 control=control,
                                 n_attacks=min(2000, len(control)),
                                 aux_cols=aux_cols,
                                 n_neighbors=10)

evaluator5.evaluate(n_jobs=-2)  # n_jobs follow joblib convention. -1 = all cores, -2 = all execept one1
#evaluator.evaluate(label = 1)
evaluator5.evaluate()
evaluator5.risk()

C:\Users\26069\anaconda3\envs\myenv3.11\Lib\site-packages\anonymeter\stats\confidence.py:218: UserWarning: Attack is as good or worse as baseline model. Estimated rates: attack = 0.002455648069704588, baseline = 0.004950855451501456. Analysis results cannot be trusted.
  self._sanity_check()


PrivacyRisk(value=0.0, ci=(0.0, 0.002089093912743931))

In [51]:
res5 = evaluator5.results()

print("Successs rate of main attack:", res5.attack_rate)
print("Successs rate of baseline attack:", res5.baseline_rate)
print("Successs rate of control attack:", res5.control_rate)

Successs rate of main attack: SuccessRate(value=0.002455648069704588, error=0.0019453844860661056)
Successs rate of baseline attack: SuccessRate(value=0.004950855451501456, error=0.0029226088445362865)
Successs rate of control attack: SuccessRate(value=0.003453731022423335, error=0.00238542229400942)


In [52]:
# #Diff no DP
# evaluator6 = LinkabilityEvaluator(ori=ori, 
#                                  syn=syn, 
#                                  control=control,
#                                  n_attacks=min(2000, len(control)),
#                                  aux_cols=aux_cols,
#                                  n_neighbors=10)

# evaluator6.evaluate(n_jobs=-2)  # n_jobs follow joblib convention. -1 = all cores, -2 = all execept one
# #evaluator.evaluate(label = 1)
# #evaluator7.evaluate()
# print(evaluator6.risk())

# res7 = evaluator6.results() 

# print("Successs rate of main attack:", res6.attack_rate)
# print("Successs rate of baseline attack:", res6.baseline_rate)
# print("Successs rate of control attack:", res6.control_rate)


As visible, the attack is not very successful and the linkability risk is low. The `n_neighbor` parameter can be used to allow for weaker indirect links to be scored as successes. It will have an impact on the risk estimate. To check the measured risk for different values of `n_neighbor` you don't have to re-run the evaluation. Rather, do:

In [53]:
#print(evaluator6.risk(n_neighbors=7))

# Measuring the Inference Risk

Finally, `anonymeter` allows to measure the inference risk. It does so by measuring the success of an attacker that tries to discover the value of some secret attribute for a set of target records on which some auxiliary knowledge is available.

Similar to the case of the `LinkabilityEvaluator`, the main parameter here is `aux_cols` which specify what the attacker knows about its target, i.e. which columns are known to the attacker. By selecting the `secret` column, one can identify which attributes, alone or in combinations, exhibit the largest risks and thereby expose a lot of information on the original data.

In the following snippet we will measure the inference risk for each column individually, using all the other columns as auxiliary information to model a very knowledgeable attacker. 

In [54]:
# columns = ori.columns
# results = []

# for secret in columns:
    
#     aux_cols = [col for col in columns if col != secret]
    
#     evaluator = InferenceEvaluator(ori=ori, 
#                                    syn=syn, 
#                                    control=control,
#                                    aux_cols=aux_cols,
#                                    secret=secret,
#                                    n_attacks=1000)
#     #evaluator.evaluate(n_jobs=-2)
#     evaluator.evaluate()
#     results.append((secret, evaluator.results()))

In [55]:
ori

,city,series_dev,emui_dev,device_name,device_size,net_type,creat_type_cd,slot_id,spread_app_id,app_second_class,...,task_id_count,adv_id_count,user_id_task_id_nunique,user_id_adv_prim_id_nunique,user_id_slot_id_nunique,user_id_spread_app_id_nunique,age_task_id_nunique,age_adv_id_nunique,gender_task_id_nunique,label
3519005,213,34,20,137,2032,7,8,38,162,23,...,412,412,81,27,13,16,7019,7834,10244,0
2291869,372,31,21,151,2117,7,8,59,162,14,...,1939,1939,92,37,12,17,7019,7834,6898,0
2675847,249,30,28,278,1555,7,8,50,174,18,...,942,942,100,37,14,15,7019,7834,10244,0
1149062,186,32,32,187,1186,7,8,67,336,17,...,8,3,70,22,14,11,8363,9358,10244,0
3010903,263,30,35,113,1983,6,8,17,181,20,...,53,53,45,18,9,8,7845,8730,10244,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234658,170,31,21,151,2451,6,8,17,292,23,...,93,93,28,15,7,7,8065,9014,10244,0
1238546,437,12,19,203,1072,4,3,47,191,16,...,5182,5182,24,17,9,10,5066,5546,6898,0
289944,354,11,36,299,2032,7,8,28,162,14,...,659,659,91,39,15,17,8363,9358,10244,0
2984016,203,16,28,240,2431,7,8,65,240,29,...,4517,4517,63,43,23,19,8065,9014,10244,0


In [56]:
#with dp
ori['city'] = ori['city'].astype(int)
syn_dp['city'] = syn_dp['city'].astype(int)
control['city'] = control['city'].astype(int)

secret = 'city'

evaluator4 = InferenceEvaluator(
    ori=ori,
    syn=syn_dp,
    control=control,
    aux_cols=aux_cols,
    secret=secret,
    n_attacks=1000
)

evaluator4.evaluate()
evaluator4.risk()
res4 = evaluator4.results()
print(res4.risk())
print("Success rate of main attack:", res4.attack_rate)
print("Success rate of baseline attack:", res4.baseline_rate)
print("Success rate of control attack:", res4.control_rate)

PrivacyRisk(value=0.0, ci=(0.0, 0.21553033399586852))
Success rate of main attack: SuccessRate(value=0.9682014235117891, error=0.010704837659376213)
Success rate of baseline attack: SuccessRate(value=0.1114924358093665, error=0.019413063137509394)
Success rate of control attack: SuccessRate(value=0.9821478488929912, error=0.007979909104731292)


In [27]:
# #without dp
# dif_syn['Revenue'] = dif_syn['Revenue'].astype(int)

# secret = 'Revenue'

# evaluator8 = InferenceEvaluator(
#     ori=ori,
#     syn=dif_syn,
#     control=control,
#     aux_cols=aux_cols,
#     secret=secret,
#     n_attacks=1000
# )

# evaluator8.evaluate()
# evaluator8.risk()
# res8 = evaluator8.results()
# print(res8.risk())
# print("Success rate of main attack:", res8.attack_rate)
# print("Success rate of baseline attack:", res8.baseline_rate)
# print("Success rate of control attack:", res8.control_rate)